In [ ]:
! pip install pytorch-lightning
! pip install torchmetrics

In [8]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import pdb
import pytorch_lightning as pl
import torchmetrics 
from torchmetrics.functional import accuracy

In [31]:
class ResNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)

    self.loss = nn.CrossEntropyLoss()
    self.accuracy = torchmetrics.Accuracy()
  
  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 +h1)
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimizer = optim.SGD(self.parameters(), lr=1e-2)
    return optimizer

# == ESTE CODIGO ES SOLO PARA MULTI GPU ==
  # para entrenar con multi gpu a los parametros de pl.Trainer se suman
  # gpus=8, num_nodes=32
  # num_nodes es la cantidad de compus con 8 gpus que tenemos

  def prepare_data(self):
    # esto solo se va a ejecutar una vez, no en todas las gpu
    # aca hace lo que quieras que pase una vez
    # este metodo tiene sentido si todos los nodos comparten el mismo file system
    # si todos los nodos tienen su propio file system, lightning lo va a autodetectar y ejecutar este metodo por cada uno de ellos
    datasets.MNIST('data', train=True, download=False, transform=transforms.ToTensor())

  def setup(self):
    # aca podemos girar, normalizar imagenes, hacer pre processing del dataset 

    # setup se corre en cada gpu asi que podemos asignar estado cuando entrenas en multi-gpu estas poniendo una copia del modelo en cada gpu
    dataset = datasets.MNIST('data', train=True, download=False, transform=transforms.ToTensor())
    self.train, self.val = random_split(dataset, [55000, 5000])
# ==== 

  def train_dataloader(self):
    # esto acopla la data al modelo lo cual no siempre es deseable 
    # ponele que estas entrenando un imageNet para clasificar 5 clases
    # esta bueno que eso este definido en el modelo, queda lindo y limpio


    train_loader = DataLoader(self.train, batch_size=32)
    return train_loader

  def val_dataloader(self):
    # estos metodos no se llaman cuando se instancia la clase
    # lightning implementa lazy loading, solo cuando se ejecuta el loop de validacion, se ejecuta el data loader.

    # esto acopla la data al modelo lo cual no siempre es deseable 
    # ponele que estas entrenando un imageNet para clasificar 5 clases
    # esta bueno que eso este definido en el modelo, queda lindo y limpio
    val_loader = DataLoader(self.val, batch_size=32)
    return val_loader

  def training_step(self, batch):
    x, y = batch
    # x = batchsize * 1 * 28 * 28
    b = x.size(0)
    x = x.view(b, -1)
    # 1) forward
    logits = self(x)
    # 2) compute the objective function
    J = self.loss(logits, y)
    self.accuracy(logits, y)
    self.log(f"train_acc_step", self.accuracy, on_epoch=True)
    return {'loss': J}

  def training_epoch_end(self, batch_parts):
    acc = self.accuracy.compute()
    self.log('train_acc_epoch', acc)
    



  def validation_step(self, batch, batch_idx):
   results = self.training_step(batch)
  
  def validation_epoch_end(self, validation_step_outputs):
    avg_val_loss = torch.tensor([x['loss'] for x in validation_step_outputs]).mean()
    return

 

model = ResNet()

In [32]:
trainer = pl.Trainer(max_epochs=5, gpus=1)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/gocandra/workspace/lightning/lightning_logs


TypeError: 'method' object is not iterable

In [ ]:
class ImageClassifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.resnet = ResNet()
